In [1]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pyarrow as pa
import pyarrow.parquet as pq


In [2]:
df = pd.read_csv('/Users/gabrielrojas/Documents/PROYECTOS/PROGRAMACIÓN/HENRY/DATA-FT-20/PI/PI_MLOps_steam_recs/data/processed/user_reviews_processed.csv', engine='python')
df.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250.0,True,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200.0,True,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110.0,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014-06-24,251610.0,True,I know what you think when you see this title ...
4,js41637,2013-09-08,227300.0,True,For a simple (it's actually not all that simpl...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49077 entries, 0 to 49076
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    49077 non-null  object 
 1   posted     49070 non-null  object 
 2   item_id    49070 non-null  float64
 3   recommend  49070 non-null  object 
 4   review     49043 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.9+ MB


In [4]:
analyzer = SentimentIntensityAnalyzer()


In [5]:
def analyze_sentiment(text):
  """
  Analyzes the sentiment of a given text and returns a score and category.

  Args:
    text: The text to analyze.

  Returns:
    A tuple containing:
      - score: A float representing the sentiment score (-1 for negative, 0 for neutral, 1 for positive).
      - category: An integer representing the sentiment category (0 for negative, 1 for neutral, 2 for positive).
  """
  if not text or pd.isna(text):  # Check for missing or null values
    return 0, 1  # Neutral sentiment for missing reviews
  scores = analyzer.polarity_scores(text)
  score = scores["compound"]
  category = 0 if score < -0.05 else (2 if score > 0.05 else 1)
  return score, category


In [6]:
df["sentiment_analysis"] = df["review"].apply(analyze_sentiment)
df["sentiment_score"], df["sentiment_category"] = zip(*df["sentiment_analysis"])


In [7]:
df

,user_id,posted,item_id,recommend,review,sentiment_analysis,sentiment_score,sentiment_category
0,76561197970982479,2011-11-05,1250.0,True,Simple yet with great replayability. In my opi...,"(0.8481, 2)",0.8481,2
1,76561197970982479,2011-07-15,22200.0,True,It's unique and worth a playthrough.,"(0.2263, 2)",0.2263,2
2,76561197970982479,2011-04-21,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,"(0.9117, 2)",0.9117,2
3,js41637,2014-06-24,251610.0,True,I know what you think when you see this title ...,"(0.9566, 2)",0.9566,2
4,js41637,2013-09-08,227300.0,True,For a simple (it's actually not all that simpl...,"(0.9708, 2)",0.9708,2
...,...,...,...,...,...,...,...,...
49072,llDracuwulf,2015-10-14,730.0,True,its FUNNNNNNNN,"(0.0, 1)",0.0000,1
49073,76561198223837952,2015-10-10,253980.0,True,Awesome fantasy game if you don't mind the gra...,"(0.765, 2)",0.7650,2
49074,76561198229845636,2015-10-31,730.0,True,Prettyy Mad Game,"(-0.4939, 0)",-0.4939,0
49075,76561198232478272,2015-12-14,730.0,True,AMAZING GAME 10/10,"(0.6739, 2)",0.6739,2


In [8]:
df.drop(columns=['sentiment_analysis', 'sentiment_score'], axis=1, inplace=True)

In [9]:
df.head()

,user_id,posted,item_id,recommend,review,sentiment_category
0,76561197970982479,2011-11-05,1250.0,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,2011-07-15,22200.0,True,It's unique and worth a playthrough.,2
2,76561197970982479,2011-04-21,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,2014-06-24,251610.0,True,I know what you think when you see this title ...,2
4,js41637,2013-09-08,227300.0,True,For a simple (it's actually not all that simpl...,2


In [10]:
df.to_csv('/Users/gabrielrojas/Documents/PROYECTOS/PROGRAMACIÓN/HENRY/DATA-FT-20/PI/PI_MLOps_steam_recs/data/processed/user_reviews_processed-fe.csv', index=False)

In [11]:
pq.write_table(pa.Table.from_pandas(df), '../data/df_reviews.parquet')